<h3> prøver å gjenskape henning sin 143.</h3>
<p><p>

In [1]:
%load_ext autoreload

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%autoreload
from pipeline import Pipeline
pipin = Pipeline()

Current working directory: /Users/miksx/GitHub/Forest-Gump/mikael/autoML/gluon


In [4]:
PATH = "ag_144_exstract_time_3_12"

In [5]:
df1_0 = pipin.get_data("A")
df2_0 = pipin.get_data("B")
df3_0 = pipin.get_data("C")

In [6]:
numeric_columns = df1_0.select_dtypes(include='number').columns

# Select columns with negative values, excluding datetime columns
negative_columns = numeric_columns[(df1_0[numeric_columns] < 0).any()]
negative_columns

Index(['sun_elevation:d', 'sun_elevation:d_45'], dtype='object')

In [7]:
# remove_consecutive_measurments
# 6/24
# dette skal prøve å gjenskape henning sin
df1_0 = pipin.remove_consecutive_measurments_new(df1_0, 3, 12)
df2_0 = pipin.remove_consecutive_measurments_new(df2_0, 3, 12)
df3_0 = pipin.remove_consecutive_measurments_new(df3_0, 3, 12)

/Users/miksx/GitHub/Forest-Gump/mikael/autoML/gluon/pipeline.py:252: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['consecutive_group'] = df.groupby(
/Users/miksx/GitHub/Forest-Gump/mikael/autoML/gluon/pipeline.py:255: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["is_first_in_consecutive_group"] = False
/Users/miksx/GitHub/Forest-Gump/mikael/autoML/gluon/pipeline.py:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

In [8]:
print(df1_0.shape, df2_0.shape, df3_0.shape)

(28405, 185) (19779, 185) (16819, 185)


In [9]:
df1_0.describe()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,super_cooled_liquid_water:kgm2_45,t_1000hPa:K_45,total_cloud_cover:p_45,visibility:m_45,wind_speed_10m:ms_45,estimated_45,observed_45,day_of_year_45,hour_45,pv_measurement
count,28398.000000,28398.000000,21599.000000,2.839800e+04,28398.000000,25705.000000,28398.000000,28398.000000,28398.000000,2.839800e+04,...,28398.000000,28398.000000,28398.000000,28398.000000,28398.000000,28398.000000,28398.000000,28398.000000,28398.000000,28405.000000
mean,6.774164,1.247887,2965.184326,6.616430e+05,183.861374,1716.823608,0.040003,277.130371,50.524544,1.816869e+05,...,0.056205,281.027374,73.299736,32215.990234,3.012018,0.107296,0.892704,178.688887,11.456828,756.691448
std,2.790600,0.035813,2617.824707,8.910664e+05,248.142624,1848.394043,0.195969,6.640284,65.809540,2.341651e+05,...,0.105999,6.505377,34.046993,17595.304688,1.753547,0.309495,0.309495,90.771195,6.724242,1239.248231
min,0.700000,1.145000,27.799999,0.000000e+00,0.000000,27.900000,0.000000,250.899994,0.000000,0.000000e+00,...,0.000000,259.000000,0.000000,265.299988,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,4.500000,1.223000,1099.900024,0.000000e+00,0.000000,560.900024,0.000000,272.399994,0.000000,0.000000e+00,...,0.000000,276.200012,51.500000,16139.324219,1.600000,0.000000,1.000000,99.000000,6.000000,0.000000
50%,6.500000,1.245000,1918.599976,1.249152e+05,34.599998,1105.900024,0.000000,277.799988,17.900000,6.555970e+04,...,0.000000,281.200012,92.599998,35709.046875,2.700000,0.000000,1.000000,181.000000,11.000000,92.180000
75%,8.700000,1.271000,4088.800049,1.186012e+06,333.675018,2057.899902,0.000000,282.200012,84.599998,3.041082e+05,...,0.100000,285.399994,99.900002,47349.250000,4.100000,0.000000,1.000000,255.000000,17.000000,969.320000
max,17.500000,1.427000,12307.700195,3.004693e+06,834.799988,11688.700195,1.000000,293.799988,336.700012,1.182250e+06,...,1.200000,303.299988,100.000000,70518.203125,11.100000,1.000000,1.000000,366.000000,23.000000,5733.420000


In [10]:
train1 = TabularDataset(df1_0)
train2 = TabularDataset(df2_0)
train3 = TabularDataset(df3_0)

In [11]:
predictor1 = TabularPredictor(label="pv_measurement", eval_metric='mean_absolute_error',
                              path=PATH+"A").fit(
    train1,
    time_limit=60,
    presets='best_quality',
    # tuning_data = tuning1,
    # use_bag_holdout=True,

    # verbosity = 3
    )

Presets specified: ['best_quality']
/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/core/utils/utils.py:564: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "ag_144_experimental_4_17A"
AutoGluon Version:  0.8.3b20231108
Python Version:     3.10.12
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:41:43 PDT 2023; root:xnu-10002.1.13~1/RELEASE_ARM64_T6000
Disk Space Avail:   235.08 GB / 494.38 GB (47.5%)
Train Data Rows:    28405
Train Data Columns: 184
Label Column: pv_measurement
Preprocessing data ...
/Users/miksx/.pyenv/version

In [12]:
test1 = pipin.get_test_data("A")
test_data1 = TabularDataset(test1)


pred1 = pd.DataFrame(predictor1.predict(test_data1))

In [13]:
predictor2 = TabularPredictor(label="pv_measurement", eval_metric='mean_absolute_error',
                              path=PATH+"B").fit(
    train2,

    presets='best_quality', 
    # hyperparameters='very_large', 
    time_limit=60,
    # tuning_data = tuning2,
    # use_bag_holdout=True,
    # verbosity = 3
    )

Presets specified: ['best_quality']
/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/core/utils/utils.py:564: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "ag_144_experimental_4_17B"
AutoGluon Version:  0.8.3b20231108
Python Version:     3.10.12
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:41:43 PDT 2023; root:xnu-10002.1.13~1/RELEASE_ARM64_T6000
Disk Space Avail:   235.02 GB / 494.38 GB (47.5%)
Train Data Rows:    19779
Train Data Columns: 184
Label Column: pv_measurement
Preprocessing data ...
/Users/miksx/.pyenv/version

In [14]:
predictor3 = TabularPredictor(label="pv_measurement", eval_metric='mean_absolute_error',
                              path=PATH+"C").fit(
    train3,

    presets='best_quality', 
    # hyperparameters='very_large', 
    time_limit=60,
    # tuning_data = tuning3,
    # use_bag_holdout=True,
    # verbosity = 3
    )

Presets specified: ['best_quality']
/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/core/utils/utils.py:564: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "ag_144_experimental_4_17C"
AutoGluon Version:  0.8.3b20231108
Python Version:     3.10.12
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:41:43 PDT 2023; root:xnu-10002.1.13~1/RELEASE_ARM64_T6000
Disk Space Avail:   235.04 GB / 494.38 GB (47.5%)
Train Data Rows:    16819
Train Data Columns: 184
Label Column: pv_measurement
Preprocessing data ...
/Users/miksx/.pyenv/version

In [15]:
test1 = pipin.get_test_data("A")
test2 = pipin.get_test_data("B")
test3 = pipin.get_test_data("C")

In [16]:
test_data1 = TabularDataset(test1)
test_data2 = TabularDataset(test2)
test_data3 = TabularDataset(test3)

# pred1 = pd.DataFrame(predictor1.predict(test_data1))
pred2 = pd.DataFrame(predictor2.predict(test_data2))
pred3 = pd.DataFrame(predictor3.predict(test_data3))

negatives_pred1 = pred1[pred1["pv_measurement"] < 0]
negatives_pred2 = pred2[pred2["pv_measurement"] < 0]
negatives_pred3 = pred3[pred3["pv_measurement"] < 0]
neg = pd.concat([negatives_pred1, negatives_pred2, negatives_pred3])
neg.shape

(142, 1)

In [17]:
pred = pd.concat([pred1, pred2, pred3])
final_prediction = pipin.post_processing(pred, prediction_column="pv_measurement")
final_prediction.to_csv('submissions/gluon_3_exstract_time_3_12.csv')

In [18]:
diff = pipin.compare_mae(final_prediction)
diff

36.969885771833326

In [19]:
# predictor.evaluate(df1, silent=True)